In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/compl/complaints.csv.zip/'

In [ ]:
%cd /content/drive/MyDrive/compl/


/content/drive/MyDrive/compl


In [ ]:
!unzip complaints.csv.zip


Archive:  complaints.csv.zip
replace complaints.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/compl/complaints.csv',delimiter=',',encoding='latin-1')
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2023-08-24,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,NaN,NaN,Experian Information Solutions Inc.,NJ,07024,NaN,Other,Web,2023-08-24,In progress,Yes,NaN,7452639
1,2023-08-25,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"SANTANDER HOLDINGS USA, INC.",FL,33972,NaN,NaN,Web,2023-08-25,Closed with explanation,Yes,NaN,7512160
2,2023-07-13,Checking or savings account,Checking account,Problem caused by your funds being low,Overdrafts and overdraft fees,Citibank allowed debit card transactions to ov...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,XXXXX,NaN,Consent provided,Web,2023-07-13,Closed with explanation,Yes,NaN,7244354
3,2023-09-04,"Money transfer, virtual currency, or money ser...",Mobile or digital wallet,Trouble accessing funds in your mobile or digi...,NaN,NaN,NaN,"Paypal Holdings, Inc",NC,27587,NaN,NaN,Web,2023-09-04,Closed with explanation,Yes,NaN,7493283
4,2023-09-13,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Old information reappears or never goes away,NaN,NaN,"EQUIFAX, INC.",FL,33805,NaN,NaN,Web,2023-09-13,In progress,Yes,NaN,7537220


Dropping the unnecesary columns

In [ ]:
df.drop(['State', 'ZIP code', 'Tags','Consumer consent provided?','Submitted via', 'Date sent to company','Company response to consumer','Timely response?','Consumer disputed?'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4069701 entries, 0 to 4069700
Data columns (total 9 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  object
 6   Company public response       object
 7   Company                       object
 8   Complaint ID                  int64 
dtypes: int64(1), object(8)
memory usage: 279.4+ MB




*   Create input and output vectors.
*   Process the labels.



In [ ]:
df.drop(['Company public response'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4069701 entries, 0 to 4069700
Data columns (total 8 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  object
 6   Company                       object
 7   Complaint ID                  int64 
dtypes: int64(1), object(7)
memory usage: 248.4+ MB


In [ ]:
df.drop(['Sub-issue','Date received','Company','Complaint ID','Sub-product','Complaint ID'], axis=1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4069701 entries, 0 to 4069700
Data columns (total 3 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Product                       object
 1   Issue                         object
 2   Consumer complaint narrative  object
dtypes: object(3)
memory usage: 93.1+ MB


In [ ]:
column_name = "Product"
df = df.dropna(subset=[column_name])

In [ ]:
print(df.columns)



Index(['Product'], dtype='object')


In [ ]:
column_names = "Consumer complaint narrative"
df = df.dropna(subset=[column_names])

KeyError: ignored

In [ ]:
cats = {
    'Product': ['Bank account or service', 'Checking or savings account', 'Consumer Loan', 'Credit card', 'Credit card or prepaid card','Credit reporting or other personal consumer reports'
    ,'Credit reporting, credit repair services, or other personal consumer reports','Debt collection','Debt or credit management','Money transfer, virtual currency, or money service'
    ,'Money transfers','Mortgage','Other financial service','Payday loan','Payday loan, title loan, or personal loan','Payday loan, title loan, personal loan, or advance loan',
                'Prepaid card','Student loan','Vehicle loan or lease'],
}

# creating a Dataframe object
df = pd.DataFrame(cats)

# Applying the condition
df.loc[df["Product"] == "Bank account or service", "Product"] = 0
df.loc[df["Product"] == "Checking or savings account", "Product"] = 0
df.loc[df["Product"] == "Consumer Loan", "Product"] = 2
df.loc[df["Product"] == "Credit card", "Product"] = 0
df.loc[df["Product"] == "Credit card or prepaid card", "Product"] = 0
df.loc[df["Product"] == "Credit reporting or other personal consumer reports", "Product"] = 0
df.loc[df["Product"] == "Credit reporting, credit repair services, or other personal consumer reports", "Product"] = 0
df.loc[df["Product"] == "Debt collection", "Product"] = 1
df.loc[df["Product"] == "Debt or credit management", "Product"] = 0
df.loc[df["Product"] == "Money transfer, virtual currency, or money service", "Product"] = 0
df.loc[df["Product"] == "Money transfers", "Product"] = 0
df.loc[df["Product"] == "Mortgage", "Product"] = 3
df.loc[df["Product"] == "Other financial service", "Product"] = 0
df.loc[df["Product"] == "Payday loan", "Product"] = 2
df.loc[df["Product"] == "Payday loan, title loan, or personal loan", "Product"] = 2
df.loc[df["Product"] == "Payday loan, title loan, personal loan, or advance loan", "Product"] = 2
df.loc[df["Product"] == "Prepaid card", "Product"] = 0
df.loc[df["Product"] == "Student loan", "Product"] = 2
df.loc[df["Product"] == "Vehicle loan or lease", "Product"] = 2


In [ ]:
X = df['Consumer complaint narrative']
Y = df.Product
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

KeyError: ignored



Split into training and test data

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

**Process the data**


*   Tokenize the data and convert the text to sequences.
*   Add padding to ensure that all the sequences have the same shape.

*   There are many ways of taking the max_len and here an arbitrary length of 150 is chosen.






In [ ]:
# Convert numeric values to strings
X_train = [str(item) for item in X_train]


In [ ]:
from keras.utils import pad_sequences

max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)


**RNN**

Define the RNN structure.


In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(128,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

Call the function and compile the model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer="adam", metrics=['accuracy'])



Fit on the training data.


In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=64,epochs=15,
          validation_split=0.1)